# PEC

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cellpy

In [ ]:
# C:\scripts\cellpy\dev_data\PEC
filename = "Test1058_2.csv"
raw_file_path = (Path("../../../../Eksempelfiler") / filename).resolve()

In [ ]:
raw_file_path.is_file()

In [ ]:
pec = cellpy.get(
    instrument="pec_csv", filename=raw_file_path, cycle_mode="cathode", mass=50_000
)

In [ ]:
import holoviews as hv

hv.extension("bokeh")

In [ ]:
pec.get_cycle_numbers()

In [ ]:
steptable = pec.cell.steps
steptable.head(10)

In [ ]:
summary = pec.cell.summary
summary.head(5)

In [ ]:
c, v = pec.get_dcap(3)

In [ ]:
fig = plt.figure(figsize=(8, 5))
curve1 = plt.scatter(c, v)  # hv.Scatter((c,v))
plt.xlabel("Discharge capacity [mAh/g]")
plt.ylabel("Voltage [V]")
plt.show()
# curve1

In [ ]:
from cellpy import log

log.setup_logging(default_level="DEBUG")

In [ ]:
from cellpy.utils import ica

In [ ]:
dv1, dq1 = ica.dqdv(
    v,
    c,
)

In [ ]:
converter = ica.Converter()
converter.pre_smoothing = False
converter.post_smoothing = False
converter.smoothing = False
converter.normalise = True
converter.voltage_fwhm = 0.01
converter.max_points = 400
converter.capacity_resolution = 5.0
converter.voltage_resolution = 0.01
converter.set_data(c, v)
converter.inspect_data()
converter.pre_process_data()
converter.increment_data()
converter.post_process_data()

fig = plt.figure(figsize=(8, 5))
curve00 = plt.plot(c, v, label="as is")
curve01 = plt.plot(
    converter.capacity_preprocessed,
    converter.voltage_preprocessed,
    label="preprocessed",
)
plt.xlabel("Discharge capacity [mAh/g]")
plt.ylabel("Voltage [V]")
plt.legend()
plt.show()

# curve00 = hv.Scatter((c, v), label="as is").opts(width=800, height=500)
# curve01 = hv.Scatter((converter.capacity_preprocessed, converter.voltage_preprocessed), label="preprocessed").opts(width=800, height=500)
# curve00 * curve01

In [ ]:
print(len(converter._voltage_processed))
print(len(converter._incremental_capacity))
fig = plt.figure(figsize=(8, 5))
curve1 = plt.plot(
    dv1,
    dq1,
    label="as is",
)
curve2 = plt.plot(
    converter.voltage_processed, converter.incremental_capacity, label="new"
)
curve3 = plt.plot(
    converter._voltage_processed, converter._incremental_capacity, label="pre"
)
curve4 = plt.plot(
    converter.voltage_processed, converter._incremental_capacity, label="shifted"
)
plt.xlabel("Voltage [V]")
plt.ylabel("dQdV [mAh/$g^{-1}V^{-1}$]")
plt.legend()
plt.show()

# curve1 = hv.Curve((dv1, dq1), label="as is").opts(width=800, height=500)
# curve2 = hv.Curve((converter.voltage_processed, converter.incremental_capacity), label="new").opts(width=800, height=500)
# curve3 = hv.Curve((converter._voltage_processed, converter._incremental_capacity), label="pre").opts(width=800, height=500)
# curve4 = hv.Curve((converter.voltage_processed, converter._incremental_capacity), label="shifted").opts(width=800, height=500)
# curve1 * curve2 * curve3 * curve4

In [ ]:
dv1, dq1 = ica.dqdv(
    v,
    c,
    voltage_fwhm=0.01,
)


dv2, dq2 = ica.dqdv(
    v,
    c,
    voltage_fwhm=0.01,
    max_points=200,
)

fig = plt.figure(figsize=(8, 5))
curve2 = plt.plot(dv1, dq1, label="as is")
scatter2 = plt.plot(dv2, dq2, label="1000 points")
plt.xlabel("Voltage [V]")
plt.ylabel("dQdV [mAh/$g^{-1}V^{-1}$]")
plt.legend()
plt.show()

# curve2 = hv.Curve((dv1, dq1), label="as is").opts(width=800, height=500)
# scatter2 = hv.Curve((dv2, dq2), label="1000 points").opts(width=800, height=500)
# curve2 * scatter2

In [ ]:
?ica.dqdv